In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from repeng.train.train_adapter import evaluate_daily_dilemma, evaluate_model, load_model, load_labels, TrainingConfig, get_choice_ids, select_dilemma_by_values, load_and_process_daily_dilemmas_eval_dataset, process_daily_dilemma_results, proj_root, register_ipissa_peft, setup_adapter, add_adapter_name_to_sd, train_steer_vector
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import torch
import pandas as pd
import gc

In [3]:

results_dir = sorted(( proj_root / "./outputs/adapters/").glob("*"))
for res_dir in results_dir:
    print(res_dir)

adapter_path = results_dir[-1] / "adapter_model.safetensors"


/media/wassname/SGIronWolf/projects5/2025/llm_moral_lb_v2/repeng/outputs/adapters/honest_contrastive_ipissa_20251108_205443
/media/wassname/SGIronWolf/projects5/2025/llm_moral_lb_v2/repeng/outputs/adapters/honest_contrastive_ipissa_20251109_062438
/media/wassname/SGIronWolf/projects5/2025/llm_moral_lb_v2/repeng/outputs/adapters/honest_contrastive_ipissa_20251109_080729
/media/wassname/SGIronWolf/projects5/2025/llm_moral_lb_v2/repeng/outputs/adapters/honest_contrastive_ipissa_20251109_133105
/media/wassname/SGIronWolf/projects5/2025/llm_moral_lb_v2/repeng/outputs/adapters/honest_contrastive_ipissa_20251109_210703
/media/wassname/SGIronWolf/projects5/2025/llm_moral_lb_v2/repeng/outputs/adapters/honest_contrastive_ipissa_20251109_222644
/media/wassname/SGIronWolf/projects5/2025/llm_moral_lb_v2/repeng/outputs/adapters/honest_contrastive_ipissa_20251109_224048
/media/wassname/SGIronWolf/projects5/2025/llm_moral_lb_v2/repeng/outputs/adapters/honest_contrastive_ipissa_20251109_225447
/media/w

In [ ]:
# load
import json, cattrs
# now evaluate with prompts
d = json.loads((results_dir[-1] / "training_config.json").read_text())
config = cattrs.structure(d, TrainingConfig)
model_id = config.model_name
base_model, tokenizer = load_model(model_id, quantization_type=config.quantization_type)

# load adapter

# Register InnerPiSSA
register_ipissa_peft()

# Load model and adapter
model = setup_adapter(base_model, config)

# model.model.load_adapter(results_dir)

2025-11-11 05:50:55.048 | INFO     | repeng.train.train_adapter:load_model:220 - Loading model: Qwen/Qwen3-4B-Instruct-2507


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
import safetensors.torch
sd = safetensors.torch.load_file(adapter_path)
sd = add_adapter_name_to_sd(sd, adapter_name='honest', prefix="ipissa_")

# now we need to add in .default ?
r = model.load_state_dict(sd, strict=False)
assert not r.unexpected_keys, f"Unexpected keys in state_dict: {r.unexpected_keys[:5]}, missing_keys {r.missing_keys[::5]}"

In [ ]:
# now evaluate with prompts
# config = TrainingConfig()
model_id = config.model_name
base_model, tokenizer = load_model(model_id, quantization_type=config.quantization_type)


choice_ids = get_choice_ids(tokenizer)
# res, df_res_pv = evaluate_model(model, tokenizer, config, dirs_pca)
generation_config = GenerationConfig(
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    bos_token_id=tokenizer.bos_token_id,
    use_cache=True,
    output_logits=True,
    return_dict_in_generate=True,
    do_sample=False,
)

In [ ]:
print([tokenizer.batch_decode(c) for c in choice_ids])


In [ ]:
config.eval_batch_size = 8
df_res_labeled, df_res_pv = evaluate_model(model, tokenizer, config)
df_res_pv.round(3)